In [9]:
import sys
!"{sys.executable}" -m pip install xarray plotly numpy pandas matplotlib


  Using cached xarray-2025.6.1-py3-none-any.whl.metadata (12 kB)
  Using cached plotly-6.2.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached narwhals-1.44.0-py3-none-any.whl.metadata (11 kB)
Using cached xarray-2025.6.1-py3-none-any.whl (1.3 MB)
Using cached plotly-6.2.0-py3-none-any.whl (9.6 MB)
Using cached narwhals-1.44.0-py3-none-any.whl (365 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Shantanoo Aher\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [11]:
!"{sys.executable}" -m pip install tqdm


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Shantanoo Aher\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [1]:
import os
import rasterio
import plotly
import matplotlib
import numpy as np
from tqdm import tqdm
from glob import glob

In [2]:
input_folder=r"C:\Users\Shantanoo Aher\OneDrive\Documents\Jul25_133189"
output_folder=r"C:\Users\Shantanoo Aher\OneDrive\Documents\INSAT_CLEANED_DATA"
npy_folder=r"C:\Users\Shantanoo Aher\OneDrive\Documents\npyf"

os.makedirs(output_folder, exist_ok=True)
os.makedirs(npy_folder, exist_ok=True)

In [3]:
Valid_ranges={
    "WV":    (0.5, 8),
    "TIR1":  (0.5, 12),
    "TIR2":  (0.5, 12),
    "SWIR":  (0,100),
    "VIS" :  (0,100),
}
BAND_ORDER = list(Valid_ranges.keys())
band_to_index = {band: i for i, band in enumerate(BAND_ORDER)}

In [15]:
wavelength_m = {
    "VIS":  0.65e-6,
    "SWIR": 1.625e-6,
    "MIR":  3.90e-6,
    "WV":   6.80e-6,
    "TIR1": 10.80e-6,
    "TIR2": 12.00e-6,
}

In [16]:
#Selective conversion of radiance into Brightness Temperature (BT)
def radiance_to_BT(L,wavelength_m):
    h = 6.62607015e-34  # Planck constant (J·s)
    c = 2.99792458e8    # Speed of light (m/s)
    k = 1.380649e-23    # Boltzmann constant (J/K)

    c1 = 2 * h * c**2
    c2 = h * c / k
    with np.errstate(divide='ignore', invalid='ignore'):
        numerator = c2 / wavelength_m
        denominator = np.log((c1 / (L * wavelength_m**5)) + 1)
        BT = numerator / denominator

    return np.nan_to_num(BT, nan=0.0, posinf=0.0, neginf=0.0)

In [22]:
def process_bands(band_data, wavelengths, valid_ranges):
    thermal_bands = ["WV", "TIR1", "TIR2"]
    processed_data = {}

    for band, radiance in band_data.items():
        # Inspect input radiance range
        min_rad = np.min(radiance)
        max_rad = np.max(radiance)
        print(f"[{band}] Input Radiance Range: min={min_rad:.3f}, max={max_rad:.3f}")

        # Get valid range
        min_valid, max_valid = valid_ranges[band]

        # Check for out-of-range values
        if min_rad < min_valid or max_rad > max_valid:
            print(f" [{band}] Radiance contains out-of-range values! Valid range: ({min_valid}, {max_valid})")

        # Clip to valid range
        clipped_radiance = np.clip(radiance, min_valid, max_valid)

        if band in thermal_bands:
            # Convert mW/cm²/sr/μm → W/m²/sr/m
            L_SI = clipped_radiance * 10 * 1e6  # (×10 → W/m²), (×1e6 → μm→m)
            wavelength = wavelengths[band]     # Already in meters
            BT = radiance_to_bt(L_SI, wavelength)
            processed_data[band] = BT
        else:
            processed_data[band] = clipped_radiance

    return processed_data


In [23]:
def process_file(file_path, band_name):
    fname = os.path.basename(file_path)

    with rasterio.open(file_path) as src:
        data = src.read(1).astype(np.float32)
        profile = src.profile
        transform = src.transform

        if not os.path.exists("lat_vals.npy"):
            width = src.width
            height = src.height
            lon_vals = np.array([transform * (x, 0) for x in range(width)])[:, 0]
            lat_vals = np.array([transform * (0, y) for y in range(height)])[:, 1]
            np.save("lat_vals.npy", lat_vals)
            np.save("lon_vals.npy", lon_vals)

        
        min_val, max_val = Valid_ranges[band_name]
        data = np.where((data < min_val) | (data > max_val), np.nan, data)
#Convert to BT
        if band_name in WAVELENGTHS:
            wavelength = WAVELENGTHS[band_name]
            data = radiance_to_BT(data, wavelength)

        cleaned_path = os.path.join(output_folder, f"cleaned_{fname}")
        profile.update(dtype=rasterio.float32, nodata=0)
        with rasterio.open(cleaned_path, 'w', **profile) as dst:
            dst.write(np.nan_to_num(data, nan=0).astype(np.float32), 1)
#Save as Numpy array
        npy_name = os.path.splitext(fname)[0] + ".npy"
        np.save(os.path.join(npy_folder, npy_name), data)

In [24]:
tif_files = sorted(glob(os.path.join(input_folder, "*.tif")))

for tif_path in tqdm(tif_files, desc="Cleaning and Converting to BT"):
    fname = os.path.basename(tif_path)
    for band in Valid_ranges:
        if band in fname.upper():
            process_file(tif_path, band)
            break
    else:
        print(f"Skipping unknown band: {fname}")


Cleaning and Converting to BT: 100%|█████████████████████████████████████████████████| 186/186 [02:52<00:00,  1.08it/s]
